In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('spam_ham_dataset.csv')
df

In [ ]:
df.info

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.isnull()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df.duplicated().sum()

In [ ]:
sns.histplot(df, x='label', discrete=True, stat='density')

In [ ]:
sns.pairplot(df, hue='label_num')

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

In [ ]:

import re
import unicodedata

from string import punctuation
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
 
# print(stopwords.words('english'))

STOPWORDS = stopwords.words('english')
STEMMER = PorterStemmer()
LEMMANTIZER = WordNetLemmatizer()

def process_text(
    text, tokenized=False, lowercase=True, remove_punctuation=True,
    remove_digit=True, remove_stopwords=True, add_stopwords=[],
    ngrams=None, use_stem_lemmantize='stem',
):
    stopwords_ = set(STOPWORDS).union(set(add_stopwords))
    stopwords_ = set([s.lower() for s in stopwords_])
    # normalize, lowercase, tokenize, remove punctuation
    tokens = unicodedata.normalize('NFKC', text)
    tokens = tokens.lower() if lowercase else tokens
    if not tokenized:
        if remove_punctuation:
            pattern = '[{}\s]+'.format(punctuation)
        else:
            pattern = '([{}]+)'.format(punctuation)
            tokens = re.sub(pattern, r' \1 ', tokens)
            pattern = '\s+'
        tokens = re.split(pattern, tokens)
    # processs
    p_tokens = list()
    for token in tokens:
        # remove digit
        if remove_digit:
            token = re.sub('\d+', '', token)
        # remove stopwords
        if remove_stopwords:
            token = '' if token.lower() in stopwords_ else token
        p_tokens.append(token)
    tokens = [t for t in p_tokens if len(t) > 0]
    # stemming or lemmantizing
    if use_stem_lemmantize == 'stem':
        tokens = [STEMMER.stem(t) for t in tokens]
    elif use_stem_lemmantize == 'lemmantize':
        tokens = [LEMMANTIZER.lemmatize(t) for t in tokens]
    # make n-gram
    if not (ngrams is None):
        ngrams_list = []
        min_ngram, max_ngram = ngrams
        for n in range(min_ngram, max_ngram + 1):
            ngrams_list.append(
                [' '.join(tokens[i: i+n])
                 for i in range(len(tokens) - n)]
            )
        tokens = [t for ngrams_l in ngrams_list for t in ngrams_l]
    return tokens

In [ ]:
# test
i = 605
text = df.loc[i, 'text']

print(process_text(text, remove_punctuation=True,
                   ngrams=(1, 2)))

In [ ]:
X = df.label
y = df.text

In [ ]:
# split data
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label_num'], test_size=0.3,
    random_state=2023
)


In [ ]:
vectorizer = CountVectorizer(
    analyzer=lambda x: process_text(x, ngrams=(1, 2),
                                    use_stem_lemmantize='stem'),
    max_df=0.85,
    min_df=1,
)

In [ ]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
# define and train model
nb_model = MultinomialNB(alpha=0.01)
nb_model.fit(X_train, y_train)


In [ ]:
label_names = sorted(df['label'].unique())
label_names

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_names))

In [ ]:
# confusion matrix
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, columns=label_names, index=label_names)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
sns.heatmap(df_cm, cmap="Blues", annot=True, fmt='.0f'); # font size